# Исследование надежности заемщиков.

# Шаг 1. Обзор данных

In [1]:
# **Обзор данных**
#### для начала необходимо открыть таблицу и изучить общую информацию о данных
#### всего в таблице 12 столцов 
#*children* — количество детей в семье
#*days_employed* — общий трудовой стаж в днях
#*dob_years* — возраст клиента в годах
#*education* — уровень образования клиента
#*education_id* — идентификатор уровня образования
#*family_status* — семейное положение
#*family_status_id* — идентификатор семейного положения
#*gender — пол клиента
#income_type — тип занятости
#debt — имел ли задолженность по возврату кредитов
#total_income — ежемесячный доход
#purpose — цель получения кредита*

import pandas as pd
status = pd.read_csv('/datasets/data.csv')
display(status)


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,-4529.316663,43,среднее,1,гражданский брак,1,F,компаньон,0,224791.862382,операции с жильем
21521,0,343937.404131,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999.806512,сделка с автомобилем
21522,1,-2113.346888,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672.561153,недвижимость
21523,3,-3112.481705,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093.050500,на покупку своего автомобиля


# Шаг 2.1 Заполнение пропусков

In [2]:
# Заполнение пропусков
### Для начала нахожу два стоблца с пропущенными значениями, это столбцы days_employed  и total_income
#### Пропуски значений возможно связаны как с отсутствием данных о ежемесячном доходе и стаже, так и с безответственностью сотрудника
####### Так как пропуски не привышают общего числа значений (и все значения коичественные),возможна замена на медианные значения
#медианные значения необходимы для качественного заполнения пропусков, т.к. для нахождения медианы мы изначально 
#должны сгруппировать данные по параметру
#status_null = status.isnull().sum()
#print(status_null)

employed_median = status.groupby('income_type')['days_employed'].transform('mean')
income_median = status.groupby('income_type')['total_income'].transform('mean')
status['days_employed'] = status['days_employed'].fillna(employed_median)
status['total_income'] = status['total_income'].fillna(income_median)
status_null = status.isnull().sum()
print(status_null)


children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64


# Шаг 2.2 Проверка данных на аномалии и исправления.

In [3]:
status.describe()
# Дни не могут быть вещественным числом, дни не могут быть отрицательными
### есть проблема с выгрузкой, возможно сотрудник некорректно вносил данные
#### также стоит обратить внимание на отклонение в столбце days_employed, т.к. оно аномально большое
#### максимальное количество детей в стодбце children - 20, что также аномально для данных
#### исправим значение -1 для количества детей согласно нашей гипотезы (т.е. заменим на 1)
status.loc[status['children'] == -1, 'children'] = 1
# проверим, что все хорошо
status.loc[status['children'] == -1]['children'].count()
# исправим опечатку в purpose
status.loc[status['purpose'] == 'ремонт жилью', 'purpose'] = 'ремонт жилья'
# проверим, что все хорошо
status.loc[status['purpose'] == 'ремонт жилью']['purpose'].count()
# исправим значение пола
status.loc[status['gender'] == 'XNA', 'gender'] = 'F'
# проверим, что все хорошо
status.loc[status['gender'] == 'XNA']['gender'].count()
# исправим значения 20 для возраста согласно нашей гипотезе
status.loc[status['children'] == 20, 'children'] = 2
# проверим, что все хорошо
status.loc[status['children'] == 20]['children'].count()


0

# Шаг 2.3. Изменение типов данных.

In [4]:
# **Заменим тип total_income на int, нам нужен конкретный целочисленный тип, а не вещественный, используем именно astype**
status['total_income'] = status['total_income'].astype('int')
status.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     21525 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      21525 non-null  int64  
 11  purpose           21525 non-null  object 
dtypes: float64(1), int64(6), object(5)
memory usage: 2.0+ MB


# Шаг 2.4. Удаление дубликатов.

In [5]:
#status.duplicated().sum()

##### аргумент drop со значением True,
##### чтобы не создавать столбец со
##### старыми значениями индексов

status = status.drop_duplicates().reset_index(drop = True)
#и проверим, что все хорошо
status.duplicated().sum()
#появление дупликатов возможно с некомпететностью сотрудников
# приведем сразу все категории к одному регистру для более удобного анализа
status['education'] = status['education'].str.lower()
status.head(10)
# аргумент drop со значением True,
# чтобы не создавать столбец со
# старыми значениями индексов
#### При вызове метода drop_duplicates() вместе с повторяющимися строками удаляются их индексы, поэтому используется с методом reset_index()
#вывод, чтобы проверить результат


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,-5623.422610,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем
7,0,-152.779569,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование
8,2,-6929.865299,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи


# Шаг 2.5. Формирование дополнительных датафреймов словарей, декомпозиция исходного датафрейма.

In [6]:
# выделим словарь для education
education_dict = status[['education_id', 'education']]
# удалим дубликаты из словаря
education_dict = education_dict.drop_duplicates().reset_index(drop=True)
# проверим содержимое
education_dict.head(10)

# аналогичным образом поступим для family_status

# выделим словарь для family_status
family_status_dict = status[['family_status_id', 'family_status']]
# удалим дубликаты из словаря
family_status_dict = family_status_dict.drop_duplicates().reset_index(drop=True)
# проверим содержимое
family_status_dict.head(10)

def make_proportion(list):
    return str(round((list.sum() / list.count()) * 100,2)) + '%'
# построим сводную таблицу для ответа на вопрос (по фрейму со значениями из словаря)
data_pivot2 = status.pivot_table(index=['family_status'], values=["debt"], aggfunc=['sum', 'count', make_proportion])
# сортируем, чтобы сразу видеть у кого ситуация лучше
data_pivot2 = data_pivot2.sort_values(by=('make_proportion', 'debt'))
data_pivot2


,sum,count,make_proportion
,debt,debt,debt
family_status,,,
вдовец / вдова,63,959,6.57%
в разводе,85,1195,7.11%
женат / замужем,931,12344,7.54%
гражданский брак,388,4163,9.32%
Не женат / не замужем,274,2810,9.75%


# Шаг 2.6. Категоризация дохода.

In [7]:
def make_income_category_by_quantile(total_income):
    if 0 <= total_income <= 30000:
        return 'E'
    if 30001 <= total_income <= 50000:
        return 'D'
    if 50001 <= total_income <= 200000:
        return 'C'
    if 200001 <= total_income <= 1000000:
        return 'B'
    if  total_income <= 1000001:
        return 'A'

status['total_income_category'] = status['total_income'].apply(make_income_category_by_quantile)

# проверим, что все ок
status['total_income_category'].value_counts()
# построим сводную таблицу для ответа на вопрос
data_pivot4 = status.pivot_table(index=['total_income_category'], values=["debt"], aggfunc=['sum', 'count', make_proportion])
# сортируем, чтобы сразу видеть у кого ситуация лучше
data_pivot4 = data_pivot4.sort_values(by=('make_proportion', 'debt'))
data_pivot4

,sum,count,make_proportion
,debt,debt,debt
total_income_category,,,
D,21,350,6.0%
B,386,5545,6.96%
C,1330,15529,8.56%
E,2,22,9.09%


# Шаг 2.7. Категоризация целей кредита.

In [8]:

def make_pay(list):
    if 'авто' in list:
        return 'операции с автомобилем'
    if 'жиль' in list or 'недвижимост' in list:
        return 'операции с недвижимостью'
    if 'свадьб' in list:
        return 'проведение свадьбы' 
    if 'образов' in list:
        return 'получение образования'
   
  
    
status['purpose_category'] = status['purpose'].apply(make_pay)

status['purpose_category'].value_counts()


# построим сводную таблицу для ответа на вопрос
data_pivot3 = status.pivot_table(index=['purpose_category'], values=["debt"], aggfunc=['sum', 'count', make_proportion])
# сортируем, чтобы сразу видеть у кого ситуация лучше
data_pivot3 = data_pivot3.sort_values(by=('make_proportion', 'debt'))
data_pivot3


,sum,count,make_proportion
,debt,debt,debt
purpose_category,,,
операции с недвижимостью,782,10814,7.23%
проведение свадьбы,186,2335,7.97%
получение образования,370,4014,9.22%
операции с автомобилем,403,4308,9.35%


In [9]:
#status.info()
# функция, что посчитает нам отношение в процентах, будем использовать для ответов на все вопросы
# функция, что посчитает нам отношение в процентах, будем использовать для ответов на все вопросы
def make_children_category(children):
    if children == 1:
        return '1 ребенок'
    if children == 2:
        return '2 ребенка'
    if children >= 3:
        return 'Много детей'
    else:
        return 'Нет детей'

status['children_category'] = status['children'].apply(make_children_category)

# проверим, что все ок
status['children_category'].value_counts()

def make_proportion(list):
    return str(round((list.sum() / list.count()) * 100,2)) + '%'

# построим сводную таблицу для ответа на вопрос
data_pivot1 = status.pivot_table(index=['children_category'], values=["debt"], aggfunc=['sum', 'count', make_proportion])
# сортируем, чтобы сразу видеть у кого ситуация лучше
data_pivot1 = data_pivot1.sort_values(by=('make_proportion', 'debt'))
data_pivot1

,sum,count,make_proportion
,debt,debt,debt
children_category,,,
Нет детей,1063,14107,7.54%
Много детей,31,380,8.16%
1 ребенок,445,4856,9.16%
2 ребенка,202,2128,9.49%


# Ответы на вопросы.

# Вопрос 1:

Есть ли зависимость между количеством детей и возвратом кредита в срок?
Есть ли зависимость между семейным положением и возвратом кредита в срок?
Есть ли зависимость между уровнем дохода и возвратом кредита в срок?
Как разные цели кредита влияют на его возврат в срок?

# Вывод :

1. Когда детей нет, вероятность возникновения задолженности чуть ниже, чем если дети есть. Что любопытно, когда детей много, задолженность возникает примерно на том же уровне. В остальных случаях вероятность просрочки выше.
2. Интересно, что вдовец/вдова и люди в разводе имеют наименьший риск просрочки. Для меня это даже удивительно. Возможно, это связно с тем, что кредит берется более осознанно. А вот самая "опасная" категория - неженатые и состоящие в гражданском браке.
3. Тут мы видим, что меньше всего просрочка у самых обеспеченных людей и у тех, у кого денег особо и нет. При чем ни первые, ни вторые не особо любят оформлять заявки на кредит. А вот самыя "опасная" группа доходов - у условного среднего класса.
4. Выгоднее всего давать кредиты на недвижимость, а автомобиль и образование - в зоне риска. 

# Общий вывод:

Самый идеальный заемщик - это человек без детей, состоящий или состоявший в официальных отношениях, имеющий уровень дохода выше среднего и берущий деньги на недвижимость.

"Разброс" просроченных выплат по каждой из категорий примерно одинаковый. В среднем, это диапазоны от 7 до 9 процентов. В остальном разница в абсолютных цифрах не такая большая.